### Smart CCTV for Change and Object Detection using Open CV
##### Shreya Todmal

#### Importing libraries

In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

#### Function for keeping connected components

In [2]:
def f_keepLargeComponents(I,th):
    R = np.zeros(I.shape)<0
    unique_labels = np.unique(I.flatten())
    for label in unique_labels:
        if label == 0:
            pass
        else:
            I2 = I==label
            if np.sum(I2)>th:
                R = R | I2
    return np.float32(255*R)

In [3]:
#pip install cvlib

In [4]:
#pip install tensorflow

#### Function for object detection

In [5]:
import cvlib as cv  
from cvlib.object_detection import draw_bbox
def f_saveSeq(C,counter,th,outPutPath): # defining a function for object detection
    if len(C)<th:
        pass
    else:
        k = 1
        for frame in C:
            imName = str(counter)+'_'+ str(k)+'.jpg'
            finalPath = os.path.join(outPutPath,imName)
            bbox,labels,conf = cv.detect_common_objects(frame)
            frame = draw_bbox(frame,bbox,labels,conf)
            cv2.imwrite(finalPath,frame)
            k += 1

#### Function for saving sequence

In [6]:
def f_displaySeq(outPutPath): # function to display the output sequence
    for imName in os.listdir(outPutPath):
        frame = cv2.imread(os.path.join(outPutPath,imName))
        frame = cv2.resize(frame,dsize=(600,400))
        cv2.imshow('Display',frame)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
    cv2.destroyAllWindows()

#### Displaying the saved sequence

In [8]:
pth = r'C:\Users\Shreya\Documents\Data Analytics Project\Open CV\Sequence_data2'
f_displaySeq(pth)

#### Model for smart CCTV

In [ ]:
#mPath = r'C:\Users\Shreya\Documents\Data Analytics Project\Open CV\GroundTruth'
imPath = r'C:\Users\Shreya\Documents\Data Analytics Project\Open CV\Campus'
#imPath = r'C:\Users\Shreya\Documents\Data Analytics Project\Open CV\ShoppingMall'
fgModel = cv2.createBackgroundSubtractorMOG2() # model for creating a background subtract
leastNumOfFrames = 5
idx = []
C = []
counter = 0
outPutPath = r'C:\Users\Shreya\Documents\Data Analytics Project\Open CV\Sequence_data2'
for imName in os.listdir(imPath):  # read each frame 
    counter += 1
    frame = cv2.imread(os.path.join(imPath,imName)) #Read every frame in sequence
    frame = cv2.resize(frame,dsize=(600,400)) # resize every frame to same size
    fgmask = fgModel.apply(frame) # applying background subtract model
    K_r = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3)) #  denoising model on each frame
    fgmask = cv2.morphologyEx(np.float32(fgmask),cv2.MORPH_OPEN,K_r) # applying the denoising model
    num_labels,labels_im = cv2.connectedComponents(np.array(fgmask>0,np.uint8)) # model for connected components
    fgmask = f_keepLargeComponents(labels_im,1000) # applying the connected frame model
    if np.sum(fgmask)>0: # loop to not consider background elements
        idx.append(counter)
        C.append(frame)
    if len(idx) >= 2 and idx[-1] > idx[-2]+1: # loop to consider connected components
        f_saveSeq(C,counter,leastNumOfFrames,outPutPath)
        idx = []
        C = []
    F = np.zeros(frame.shape,np.uint8)
    F[:,:,0],F[:,:,1],F[:,:,2] = fgmask, fgmask,fgmask # displaying the original and processed image side by side
    F2 = np.hstack((frame,F))
    cv2.imshow('Display',F2) # Display the image
    k = cv2.waitKey(5) & 0xff # press escape to break the video loop/ escape video
    if k == 27:
        break
f_saveSeq(C,counter,leastNumOfFrames,outPutPath) # saving the connected sequence
cv2.destroyAllWindows()